In [1]:
import pymssql as mssql
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import date, timedelta

In [2]:
def send_query(query):
    
    '''
    Function sending query to ODS
    Input: Query(String)
    Output: Dataframe wanted 
    '''    

    
    ods = mssql.connect(host = '128.110.13.89', 
                      user = '011553', 
                      password = 'Sino821031pac')

    odscur = ods.cursor(as_dict = True)
    odscur.execute(query)
    try:
        temp = odscur.fetchall()
    except:
        temp = []
        
    row_count = int(odscur.rowcount)
    df = pd.DataFrame(temp)
    ods.commit()
    ods.close()

    return df, row_count

In [3]:
st_query = '''
SELECT RTRIM(LTRIM(S_IDNO)) AS S_IDNO,
        CASE WHEN ISDATE(DATE_YMD) = 1 THEN CONVERT(DATE, DATE_YMD) ELSE NULL END AS MTH_DATE,
		SUM(STK_MK_AMT) AS ST_ASSET,
        SUM(MP_NET_AMT) AS MP_ASSET,
        SUM(SS_NET_AMT) AS SS_ASSET
FROM ODS.dbo.STS_ASSET_CUSTOMER_D
WHERE ISDATE(DATE_YMD) = 1 
		AND LTRIM(RTRIM(S_IDNO)) LIKE '[A-Z]%' 
		AND SUBSTRING(LTRIM(RTRIM(S_IDNO)), 2, 1) IN ('1', '2') 
		AND ISNUMERIC(RIGHT(LTRIM(RTRIM(S_IDNO)), 8)) = 1
		AND RTRIM(LTRIM(S_IDNO)) IN (SELECT DISTINCT(RTRIM(LTRIM(S_IDNO))) FROM ODS.dbo.ST_CUSTOMER)
		AND LTRIM(RTRIM(S_IDNO)) NOT IN (SELECT DISTINCT(RTRIM(LTRIM(S_IDNO))) AS S_IDNO
											FROM ODS.dbo.FU_CUSTOMER
											WHERE LTRIM(RTRIM(S_IDNO)) LIKE '[A-Z]%' 
													AND SUBSTRING(LTRIM(RTRIM(S_IDNO)), 2, 1) IN ('1', '2') 
													AND ISNUMERIC(RIGHT(LTRIM(RTRIM(S_IDNO)), 8)) = 1)
		AND LTRIM(RTRIM(S_IDNO)) NOT IN (SELECT DISTINCT(RTRIM(LTRIM(S_IDNO))) AS S_IDNO
											FROM ODS.dbo.SUB_CUSTOMER
											WHERE LTRIM(RTRIM(S_IDNO)) LIKE '[A-Z]%' 
													AND SUBSTRING(LTRIM(RTRIM(S_IDNO)), 2, 1) IN ('1', '2') 
													AND ISNUMERIC(RIGHT(LTRIM(RTRIM(S_IDNO)), 8)) = 1)
		AND LEFT(DATE_YMD, 4) = '2013'
GROUP BY RTRIM(LTRIM(S_IDNO)), DATE_YMD
'''

In [4]:
df = pd.read_csv(r'D:\Customer_Value\data\st_trading.csv')
id_list = df['S_IDNO'].unique().tolist()
del df

In [ ]:
st_df, rows = send_query(st_query)

In [ ]:
st_df['MTH_DATE'] = pd.to_datetime(st_df['MTH_DATE'])
st_df = st_df[st_df.S_IDNO.isin(id_list)]

In [ ]:
st_df.to_csv(r'D:\Customer_Value\data\st_asset1.csv', index=False)